# Regularización del modelo de regresión lineal

In [ ]:
import numpy as np # https://numpy.org/
import pandas as pd # https://pandas.pydata.org/
import seaborn as sns # https://seaborn.pydata.org/
import matplotlib.pyplot as plt # https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html
%matplotlib inline

In [ ]:
# Leer el archivo generado en el notebook de preproceso
url = 'https://raw.githubusercontent.com/JASDataCTG/Diplomado-ML/main/Modulo%202/Datasets/precio_prep_noparq.csv'
df = pd.read_csv(url, header = 0)

In [ ]:
# Crear un nuevo dataframe sin la columna precio (axis=1 -> borra columna)
X_multiple = df.drop('precio', axis = 1)

In [ ]:
y_multiple = df['precio']

In [ ]:
# Importar el modulo y la función necesaria para el modelo de regresión lineal
from sklearn.linear_model import LinearRegression

### División de los datos en set de entrenamiento y validación

In [ ]:
# Importar modulo y función necesaria para hacer una división
# aleatoria de los datos
from sklearn.model_selection import train_test_split

In [ ]:
# Obtener los cuatro conjuntos de datos necesarios para el modelo
X_train, X_test, y_train, y_test = train_test_split(X_multiple, y_multiple, test_size = 0.2, random_state = 123) 

In [ ]:
# Revisar las dimensiones de las matrices de los dataframes generados
X_train.shape, X_test.shape, y_test.shape, y_train.shape

In [ ]:
linear_model_s = LinearRegression()

In [ ]:
linear_model_s.fit(X_train, y_train)

In [ ]:
# Realizar la predicción en el conjunto de entrenamiento
y_train_s = linear_model_s.predict(X_train)

In [ ]:
# Realizar la predicción en el conjunto de validación
y_test_s = linear_model_s.predict(X_test)

In [ ]:
# Importar la función que realiza el calculo del RMSE
from sklearn.metrics import r2_score

In [ ]:
# Ver la bondad de ajuste en los datos de entrenamiento
r2_score(y_train, y_train_s)

In [ ]:
# Ver la bondad de ajuste en los datos de validación
r2_score(y_test, y_test_s)

# Regularización por el método Ridge
Este método penaliza la sumatoria de los parámetros elevados al cuadrado, reduce proporcionalmente los valores de los parámetros del modelo sin que jamás alcancen el valor de cero. El grado de penalización se controla por un hiperparámetro $\lambda$, cuado este es igual a cero no hay penalización al modelo y es equivalente a aplicar el modelo de regresión tradicional, a mayor $\lambda$ la penalización aumenta y disminuyen los valores de los predictores.

$\huge{\sum_{i=1}^{n}(y_{i}-\theta_{0}-\sum_{j=1}^p\theta_{j}x_{ij})+\lambda\sum_{j=1}^p\theta_{j}^2}$

Para el caso de la regularización utilizando gradiente de descenso, la formula queda de la siguiente forma:

$\large{\theta_{j}=(1-\frac{\alpha}{m})-\alpha\frac{1}{m}\sum_{i=1}^{m}(h_{\theta}(x^{(i)})-y^{(i)})x^{(i)}}$

In [ ]:
# Importar el modulo de preproceso
from sklearn import preprocessing

In [ ]:
# Crear un modelo para la normalización de los datos de X_train
escala = preprocessing.StandardScaler().fit(X_train)

In [ ]:
# Escalar (normalizar) el conjunto de datos de entrenamiento
X_train_esc = escala.transform(X_train)

In [ ]:
X_train_esc

In [ ]:
X_train_esc.shape

In [ ]:
# Escalar el conjunto de datos de validación
X_test_esc = escala.transform(X_test)

In [ ]:
X_test_esc

In [ ]:
X_test_esc.shape

In [ ]:
#Importar el modelo de regularización Ridge
from sklearn.linear_model import Ridge

In [ ]:
# Ajustar el modelo con un $\lambda=0.5$ (alpha = 0.5 de acuerdo a los parámetros del modelo en Python)
linear_model_r = Ridge(alpha = 0.5)
linear_model_r.fit(X_train_esc, y_train)

In [ ]:
# Calcular r2 en el conjunto de validación
r2_score(y_test, linear_model_r.predict(X_test_esc))

In [ ]:
# Importar el modulo que permite determinar las puntuaciones para diferentes valores de parámetros
from sklearn.model_selection import validation_curve

In [ ]:
validation_curve?

In [ ]:
# Crear 100 números espaciados entre con exponentes entre -2 y -8 para ser transformados
# posteriormente como los lambda de la regularización
rango_parametros = np.logspace(-2,8,100)

In [ ]:
rango_parametros

In [ ]:
# Determinar puntuaciones para entrenamiento y validación para el parámetro lambda
# (alpha en Python) para los métodos de regularización
train_scores, test_scores = validation_curve(Ridge(), X_train_esc, y_train, "alpha", rango_parametros, scoring = "r2")

In [ ]:
print(train_scores)
print('__________________________________________________________')
print(test_scores)

In [ ]:
train_scores.shape, test_scores.shape

In [ ]:
# Media de las puntuaciones de entrenamiento
train_media = np.mean(train_scores, axis = 1)

In [ ]:
train_media.shape

In [ ]:
# Media de las puntuaciones de validación
test_media = np.mean(test_scores, axis = 1)

In [ ]:
test_media.shape

In [ ]:
train_media

In [ ]:
test_media

In [ ]:
# Ver el valor máximo de las medias de las puntuaciones de validación
max(test_media)

In [ ]:
# Realizar la gráfica del rango de parámetros vs la media de las puntuaciones de validación (alpha)
sns.jointplot(x = np.log(rango_parametros), y = test_media)

In [ ]:
# Conocer el índice de la puntuación máxima de validación para determinar su correspondiente
# valor de alpha
np.where(test_media==max(test_media))

In [ ]:
# Valor de alpha que nos permite desarrollar el mejor modelo
rango_parametros[31]

In [ ]:
# Aplicar el valor de alfa que desarrolla el mejor modelo lineal basado en Ridge
best_linear_model_r = Ridge(alpha = rango_parametros[31])

In [ ]:
best_linear_model_r.fit(X_train_esc, y_train)

In [ ]:
r2_score(y_test, best_linear_model_r.predict(X_test_esc))

In [ ]:
r2_score(y_train, best_linear_model_r.predict(X_train_esc))

# Regularización por el método Lasso
Este método penaliza la sumatoria de los valores absolutos de los parámetros elevados al cuadrado, forza a que los parámetros tiendan a cero. El grado de penalización se controla por un hiperparámetro $\lambda$, cuado este es igual a cero no hay penalización al modelo y es equivalente a aplicar el modelo de regresión tradicional, a mayor $\lambda$ la penalización aumenta y se excluyen predictores.

$\sum_{i=1}^{n}(y_{i}-\theta_{0}-\sum_{j=1}^p\theta_{j}x_{ij})+\lambda\sum_{j=1}^p\|\theta_{j}\|$

### **Desarrolle la regularización Lasso y confronte los resultados con los obtenidos en el apartado anterior.**

In [ ]:
from sklearn.linear_model import Lasso